In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import optuna
import joblib

# Read data
data = pd.read_excel('TrainClass.xlsx')
real_data = pd.read_excel('TestClass.xlsx')

# Prepare training data
Y, X = data['FRACASO'], data.drop(['FRACASO', 'CODIGO_EMPRESA'], axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

# Prepare testing data
X = real_data.drop(['CODIGO_EMPRESA'], axis=1)

# Define the objective function for cross-validated AUC
def objective(trial):
    # Define hyperparameters to be optimized
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 1, 10)

    # Create the classifier with suggested hyperparameters
    clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=0)

    # Define cross-validation strategy (StratifiedKFold for classification)
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

    # Perform cross-validation and get mean AUC
    auc_scores = cross_val_score(clf, X_train, Y_train, cv=cv, scoring='roc_auc')
    mean_auc = auc_scores.mean()

    return mean_auc

# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Get the best hyperparameters
best_params = study.best_params
print(f'Best Hyperparameters: {best_params}')

# Train the final model with the best hyperparameters
best_clf = RandomForestClassifier(**best_params, random_state=0)
best_clf.fit(X_train, Y_train)

# Predictions on training set
y_pred_train = best_clf.predict(X_train)

# Calculate AUC for training set
auc_train = roc_auc_score(Y_train, y_pred_train)
print(f'AUC for Training Set: {auc_train:.4f}')

# Predictions on testing set
y_pred_test = best_clf.predict(X_test)

# Calculate AUC for testing set
auc_test = roc_auc_score(Y_test, y_pred_test)
print(f'AUC for Testing Set: {auc_test:.4f}')

# Save the trained model to a file
joblib.dump(best_clf, 'random_forest_model.joblib')

# Load model and generate probability CSV
loaded_model = joblib.load('random_forest_model.joblib')
prob = loaded_model.predict_proba(X)[:, 1]
prob_df = pd.DataFrame(prob, columns=['Probability'])
prob_df.index = prob_df.index + 1
prob_df.index.name = 'Id'
prob_df.to_csv('intento.csv', index=True)

[I 2023-12-05 14:38:29,581] A new study created in memory with name: no-name-0b52f2bd-6461-424e-a1f0-66f1fae42cd8
[I 2023-12-05 14:38:33,199] Trial 0 finished with value: 0.8434251207729468 and parameters: {'n_estimators': 93, 'max_depth': 4}. Best is trial 0 with value: 0.8434251207729468.
[I 2023-12-05 14:38:41,050] Trial 1 finished with value: 0.8378760430390866 and parameters: {'n_estimators': 185, 'max_depth': 2}. Best is trial 0 with value: 0.8434251207729468.
[I 2023-12-05 14:38:43,084] Trial 2 finished with value: 0.8360686209925341 and parameters: {'n_estimators': 50, 'max_depth': 3}. Best is trial 0 with value: 0.8434251207729468.
[I 2023-12-05 14:38:48,816] Trial 3 finished with value: 0.834903162055336 and parameters: {'n_estimators': 159, 'max_depth': 9}. Best is trial 0 with value: 0.8434251207729468.
[I 2023-12-05 14:38:50,603] Trial 4 finished with value: 0.8300507246376811 and parameters: {'n_estimators': 96, 'max_depth': 2}. Best is trial 0 with value: 0.8434251207729

Best Hyperparameters: {'n_estimators': 138, 'max_depth': 3}
AUC for Training Set: 0.6186
AUC for Testing Set: 0.5880
